In [1]:
import re
import requests
from bs4 import BeautifulSoup as bs
import unicodedata

In [32]:
def parse_10q_filing(link, section):
    
    if section not in [0, 1, 2, 3]:
        print("Not a valid section")
        sys.exit()
    
    def get_text(link):
        page = requests.get(link, headers={'User-Agent': 'Mozilla'})
        html = bs(page.content, "lxml")
        text = html.get_text()
        text = unicodedata.normalize("NFKD", text).encode('ascii', 'ignore').decode('utf8')
        text = text.split("\n")
        text = " ".join(text)
        return(text)
    
    def extract_text(text, item_start, item_end):
        item_start = item_start
        item_end = item_end
        starts = [i.start() for i in item_start.finditer(text)]
        ends = [i.start() for i in item_end.finditer(text)]
        positions = list()
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s,e])
        item_length = 0
        item_position = list()
        for p in positions:
            if (p[1]-p[0]) > item_length:
                item_length = p[1]-p[0]
                item_position = p

        item_text = text[item_position[0]:item_position[1]]

        return(item_text)

    text = get_text(link)
        
    if section == 1 or section == 0:
        try:
            item1_start = re.compile("item\s*[1][\.\;\:\-\_]*\s*\\bF", re.IGNORECASE)
            item1_end = re.compile("item\s*2[\.\;\:\-\_]\s*Man|item\s*3[\.\,\;\:\-\_]\s*Quanti", re.IGNORECASE)
            finText = extract_text(text, item1_start, item1_end)
        except:
            finText = "Something went wrong!"
        
    if section == 2 or section == 0:
        try:
            item2_start = re.compile("item\s*[2][\.\;\:\-\_]*\s*\\bM", re.IGNORECASE)
            item2_end = re.compile("item\s*3[\.\;\:\-\_]\sQuanti|item\s*4[\.\,\;\:\-\_]\s*", re.IGNORECASE)
            mdaText = extract_text(text, item2_start, item2_end)
        except:
            mdaText = "Something went wrong!"
            
    if section == 3 or section == 0:
        try:
            item3_start = re.compile("item\s*[3][\.\;\:\-\_]*\s*\\bQ", re.IGNORECASE)
            item3_end = re.compile("item\s*4[\.\;\:\-\_]\sCon|item\s*4[\.\,\;\:\-\_]\s*", re.IGNORECASE)
            riskText = extract_text(text, item3_start, item3_end)
        except:
            riskText = "Something went wrong!"        
    
    if section == 0:
        data = [finText, mdaText, riskText]
    elif section == 1:
        data = [finText]
    elif section == 2:
        data = [mdaText]
    elif section == 3:
        data = [riskText]
    return(data)

In [34]:
parse_10q_filing('https://www.sec.gov/Archives/edgar/data/0001764925/000176492520000400/work-20200430.htm',2)

['ITEM 2. MANAGEMENTS DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONSThe following discussion and analysis of our financial condition and results of operations should be read in conjunction with our condensed consolidated financial statements and related notes appearing elsewhere in this Quarterly Report on Form 10-Q and our Annual Report on Form 10-K. In addition to historical financial information, the following discussion and analysis contains forward-looking statements that are based upon current plans, expectations and beliefs that involve risks and uncertainties. Our actual results may differ materially from those anticipated in these forward-looking statements as a result of various factors, including, but not limited to, those set forth under the section titled Risk Factors in Item 1A of Part II of this Quarterly Report on Form 10-Q and in our Annual Report on Form 10-K. Our fiscal year ends January 31.OverviewSlack is a new layer of the business techno